<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
  });
  MathJax.Hub.Config({
    TeX: { extensions: ["AMSmath.js", "AMSsymbols.js", "autobold.js", "autoload-all.js"] }
  });
  MathJax.Hub.Config({
    tex2jax: {
      inlineMath: [['$','$'], ['\\(','\\)']],
      processEscapes: true
    }
  });
</script>
<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS_HTML">
</script>

# User guide

---

`TaylorSeries.jl` can be thought of as a polynomial algebraic manipulator in one or more
variables; these two cases are treated separately for now.  Three new types are defined,
`Taylor1`, `HomogeneousPolynomial` and `TaylorN`, which correspond to
expansions in one independent variable, homogeneous polynomials of various variables, and the polynomial
series in many independent variables, respectively. These types are subtypes
of `Number` and are defined parametrically.

The package is loaded as usual:

In [ ]:
using TaylorSeries

## One variable

Taylor expansions in one variable are represented by the `Taylor1` type, which
consists of a vector of coefficients (field `coeffs`) and the maximum
order considered for the expansion (field `order`). The
coefficients are arranged in ascending order with respect to the power of the
independent variable, so that
`coeffs[1]` is the constant term, `coeffs[2]` gives the first order term,
etc. This is a dense representation of the polynomial.
The order of the polynomial can be
omitted in the constructor, which is then fixed from the length of the
vector of coefficients; otherwise, the maximum
of the length of the vector of coefficients and the given integer is taken.

In [ ]:
Taylor1([1, 2, 3]) # Polynomial of order 2 with coefficients 1, 2, 3

In [ ]:
Taylor1([0.0, 1im]) # Also works with complex numbers

In [ ]:
affine(a) = a + Taylor1(typeof(a),5)  ## a + t of order 5

In [ ]:
t = affine(0.0) # Independent variable `t`

Note that the information about the maximum order considered is displayed
using a big-O notation.

The definition of `affine(a)` uses the method `Taylor1{T<:Number}(::Type{T},order::Int)`, which is a
shortcut to define the independent variable of a Taylor expansion,
with a given type and given order. As we show below, this is one of the
easiest ways to work with the package.

The usual arithmetic operators (`+`, `-`, `*`, `/`, `^`, `==`) have been
extended to work with the `Taylor1` type, including promotions that involve
`Number`s. The operations return a valid Taylor expansion with the same
maximum order; compare the last example below, where this is not possible:

In [ ]:
t*(3t+2.5)

In [ ]:
1/(1-t)

In [ ]:
t*(t^2-4)/(t+2)

In [ ]:
tI = im*t

In [ ]:
t^6  # order is 5

In [ ]:
(1-t)^3.2

In [ ]:
(1+t)^t

If no valid Taylor expansion can be computed, an error is thrown.

In [ ]:
1/t 

In [ ]:
t^3.2

Several elementary functions have been implemented; these compute their
coefficients recursively. So far, these functions are `exp`, `log`, `sqrt`, `sin`, `cos`
and `tan`;
more will be added in the future. Note that this way of obtaining the
Taylor coefficients is not the *laziest* way, in particular for many independent
variables. Yet, it is quite efficient, especially for the integration of
ordinary differential equations, which is among the applications we have in mind.

In [ ]:
exp(t)

In [ ]:
log(1-t)

In [ ]:
sqrt(t)

In [ ]:
sqrt(1 + t)

In [ ]:
imag(exp(tI)')

In [ ]:
real(exp(Taylor1([0.0,1im],17))) - cos(Taylor1([0.0,1.0],17)) == 0.0

In [ ]:
convert(Taylor1{Rational{Int}}, exp(t))

Differentiating and integrating is straightforward for polynomial expansions in
one variable. The last coefficient of a derivative is set to zero to keep the
same order as the original polynomial; for the integral, an
integration constant may be set to a different value (the default is zero). The
order of the resulting polynomial is not changed.
The $n$-th ($n \ge 0$) derivative at `t=0` is obtained using `derivative(n,a)`  where `a` is a Taylor series.

In [ ]:
derivative(exp(t))

In [ ]:
integrate(exp(t))

In [ ]:
integrate( exp(t), 1.0)

In [ ]:
integrate( derivative( exp(-t)), 1.0 ) == exp(-t)

In [ ]:
evaluate(derivative(exp(affine(1.0))),0.) == exp(1.0)  # deriv of `exp(1+t)` at t=0

In [ ]:
derivative(5, exp(affine(1.0))) == exp(1.0) # Fifth derivative of `exp(1+t) at t=0`

To evaluate a Taylor series at a point, Horner's rule is used via the function
`evaluate(a::Taylor, dt::Number)`. Here, $dt$ is the increment from
the point $t_0$ where the Taylor expansion is calculated, i.e., the series
is evaluated at $t = t_0 + dt$. Omitting $dt$ corresponds to $dt = 0$.

In [ ]:
evaluate(exp(affine(1.0))) - e # exp(t) around t0=1 (order 5), evaluated there (dt=0)

In [ ]:
evaluate(exp(t), 1) - e # exp(t) around t0=0 (order 5), evaluated at t=1

In [ ]:
evaluate( exp( Taylor1(Float64,17) ), 1) - e # exp(t) around t0=0 (order 17), evaluated at t=1
0.0

In [ ]:
tBig = Taylor1([zero(BigFloat),one(BigFloat)],50) # With BigFloats

In [ ]:
eBig = evaluate( exp(tBig), one(BigFloat) )

In [ ]:
e - eBig

## Many variables

A polynomial in $N>1$ variables can be represented in (at least) two ways:
As a vector whose coefficients are homogeneous polynomials of fixed degree, or
as a vector whose coefficients are polynomials in $N-1$ variables. We have opted
to implement the first option, which seems to show better performance. An elegant
(lazy) implementation of the second representation was discussed on the
[julia-users](https://groups.google.com/forum/#!msg/julia-users/AkK_UdST3Ig/sNrtyRJHK0AJ) list.

`TaylorN` is thus constructed as a vector of parameterized homogeneous polynomials
defined by the type `HomogeneousPolynomial`, which in turn is a vector of
coefficients of given order (degree). This implementation imposes that the user
has to specify the (maximum) order and the number of independent
variables, which is done using the `set_variables(names)` function.
`names` is a string consisting of the desired *output* names of the variables,
separated by spaces. A vector of the resulting Taylor variables is returned:

In [ ]:
x, y = set_variables("x y")

In [ ]:
x

In [ ]:
typeof(x)

In [ ]:
x.order

In [ ]:
x.coeffs

As shown, the resulting objects are of `TaylorN{Float64}` type. There is an optional `order` keyword argument for `set_variables`:

In [ ]:
set_variables("x y", order=10)

Numbered variables are also available by specifying a single
variable name and the optional keyword argument `numvars`:

In [ ]:
set_variables("α", numvars=3)

The function ``show_params_TaylorN()` displays the current values of the parameters, in an info block.

In [ ]:
show_params_TaylorN()

Internally, changing these parameters defines dictionaries that
translate the position of the coefficients of a `HomogeneousPolynomial`
into the corresponding
multi-variable monomials. Fixing these values from the start is imperative.

The easiest way to construct a `TaylorN` object is by defining symbols for
the independent variables, as above. Again, the Taylor expansions are implemented
around 0 for all variables; if the expansion
is needed around a different value, the trick is a simple translation of
the corresponding
independent variable $x \to x+a$.


Other ways of constructing `TaylorN` polynomials involve using `HomogeneousPolynomial`
objects directly, which is uncomfortable:

In [ ]:
set_variables("x", numvars=2);

In [ ]:
HomogeneousPolynomial([1,-1])

In [ ]:
TaylorN( [HomogeneousPolynomial([1,0]), HomogeneousPolynomial([1,2,3])], 4)

As before, the usual arithmetic operators (`+`, `-`, `*`, `/`, `^`, `==`)
have been extended to work with `TaylorN` objects, including the appropriate
promotions to deal with numbers. (Some of the arithmetic operations have
also been extended for
`HomogeneousPolynomial`, whenever the result is a `HomogeneousPolynomial`;
division, for instance, is not extended.) Also, the elementary functions have been
implemented, again by computing their coefficients recursively:

In [ ]:
x, y = set_variables("x y", order=10);

In [ ]:
exy = exp(x+y)

The function `getcoeff(a,v)`
gives the coefficient of `a::TaylorN` that corresponds to the monomial
specified by the vector of powers `v`:

In [ ]:
typeof(exy)

In [ ]:
rationalize(getcoeff(exy, [3,5]))

Partial differentiation is also implemented for `TaylorN` objects,
through `derivative`; integration is yet to be implemented.

In [ ]:
f(x,y) = x^3 + 2x^2 * y - 7x + 2

In [ ]:
g(x,y) = y - x^4

In [ ]:
derivative( f(x,y), 1 )   # partial derivative with respect to 1st variable

In [ ]:
derivative( g(x,y), 2 )

In [ ]:
derivative( g(x,y), 3 )   # error, since we are dealing with 2 variables

`evaluate` can also be used for `TaylorN` objects, using it on vectors of
numbers (`Real` or `Complex`); the length of the vector must coincide with the number
of independent variables.

In [ ]:
evaluate(exy, [.1,.02]) == e^0.12

Functions to compute the gradient, Jacobian and
Hessian have also been implemented. Using the
functions $f(x,y) = x^3 + 2x^2 y - 7 x + 2$ and $g(x,y) = y-x^4$ defined above,
we may use `gradient` or `∇` (`\nabla+TAB`); the results are of
type `Array{TaylorN{T},1}`. To compute the Jacobian or Hessian of a vector field
evaluated at a point, we use `jacobian` and `hessian`:

In [ ]:
f1 = f(x,y)

In [ ]:
g1 = g(x,y)

In [ ]:
gradient( g1 )

In [ ]:
∇(f1)

In [ ]:
fg = f1-g1-2*f1*g1

In [ ]:
jacobian([f1,g1], [2,1])

In [ ]:
hessian(fg, [1.0,1.0])

## Examples

### 1. Four-square identity

Euler proved the [following four-square identity](https://en.wikipedia.org/wiki/Euler%27s_four-square_identity):

\begin{eqnarray*}
(a_1^2+a_2^2+a_3^2+a_4^2) \cdot (b_1^2+b_2^2+b_3^2+b_4^2) = & &
   (a_1 b_1 + a_2 b_2 + a_3 b_3 + a_4 b_4)^2  \\
  &+& (a_1 b_2 - a_2 b_1 + a_3 b_4 -a_4 b_3)^2  \\
  &+& (a_1 b_3 - a_2 b_4 - a_3 b_1 +a_4 b_2)^2  \\
  &+& (a_1 b_4 + a_2 b_3 - a_3 b_2 -a_4 b_1)^2.
\end{eqnarray*}

The following code checks this; it can also be found in the test suite for the package.

We first define the variables:

In [ ]:
# Define the variables α₁, ..., α₄, β₁, ..., β₄
make_variable(name, index::Int) = string(name, TaylorSeries.subscriptify(index))


In [ ]:
variable_names = [make_variable("α", i) for i in 1:4];

append!(variable_names, [make_variable("β", i) for i in 1:4]);


In [ ]:
# Create the Taylor objects (order 4, numvars=8)
a1, a2, a3, a4, b1, b2, b3, b4 = set_variables(variable_names, order=4);    

In [ ]:
a1

In [ ]:
b1

Now we compute each term appearing in the above equation, and compare them

In [ ]:
# left-hand side
lhs1 = a1^2 + a2^2 + a3^2 + a4^2;
lhs2 = b1^2 + b2^2 + b3^2 + b4^2;
lhs = lhs1 * lhs2

In [ ]:
# right-hand side
rhs1 = (a1*b1 + a2*b2 + a3*b3 + a4*b4)^2;
rhs2 = (a1*b2 - a2*b1 + a3*b4 - a4*b3)^2;
rhs3 = (a1*b3 - a2*b4 - a3*b1 + a4*b2)^2;
rhs4 = (a1*b4 + a2*b3 - a3*b2 - a4*b1)^2; 
rhs = rhs1 + rhs2 + rhs3 + rhs4

In [ ]:
lhs == rhs

The identity is thus satisfied.

### 2. Fateman test

Richard J. Fateman, from Berkeley, proposed as a stringent test
of polynomial multiplication
the evaluation of $s \cdot (s+1)$, where $s := (1+x+y+z+w)^{20}$. This is
implemented in
the function `fateman1` below. We also evaluate the alternative form $s^2+s$ in `fateman2`,
which involves fewer operations (and makes a fairer comparison to what
Mathematica does). Below we have used Julia v0.4.

In [ ]:
set_variables("x", numvars=4, order=40);

In [ ]:
function fateman1(degree::Int)
    T = Int128
    oneH = HomogeneousPolynomial(one(T), 0)
    # s = 1 + x + y + z + w
    s = TaylorN( [oneH, HomogeneousPolynomial([one(T),one(T),one(T),one(T)],1)], degree )
    s = s^degree  
    # s is converted to order 2*ndeg
    s = TaylorN(s, 2*degree)
    s * ( s+TaylorN(oneH, 2*degree) )
end

In [ ]:
@time f1 = fateman1(0) #for compilation; 

In [ ]:
@time f1 = fateman1(20);

Another implementation of the same:

In [ ]:
function fateman2(degree::Int)
    T = Int128
    oneH = HomogeneousPolynomial(one(T), 0)
    # s = 1 + x + y + z + w
    s = TaylorN( [oneH, HomogeneousPolynomial([one(T),one(T),one(T),one(T)],1)], degree )
    s = s^degree
    # s is converted to order 2*ndeg
    s = TaylorN(s, 2*degree)
    return s^2 + s
end

In [ ]:
@time f2 = fateman2(0) #for compilation;

In [ ]:
@time f2 = fateman2(20);

In [ ]:
getcoeff(f2,[1,6,7,20]) # coef x^1 y^6 z^7 w^{20}

In [ ]:
sum(TaylorSeries.size_table) # number of distinct monomials

The tests above show the necessity of using `Int128`, that
`fateman2` is about twice as fast as `fateman1`, and that the series has 135751 monomials on 4 variables.

Mathematica version 10.3.0 requires 7.606437 seconds. Then,
with TaylorSeries v0.3.0, our implementation of `fateman1` is about a factor ~2.7 faster,
and `fateman2` is a factor 5.8 faster. (The original test by Fateman
corresponds to `fateman1`, which avoids optimizations in `^2`.)